# 🏁 Quick Start: EfficientNet Fine-tuning on CPU

In [7]:
import os
from pathlib import Path
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.optim import Adam
from sklearn.metrics import classification_report
import timm


In [8]:
# Config
data_root = Path("../../../data/out_data_split")
train_dir = data_root / "train"
val_dir = data_root / "val"
batch_size = 32
num_epochs = 20  # Keep it short for first run
learning_rate = 1e-4
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)


Using device: mps


In [9]:
train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder(str(train_dir), transform=train_transforms)
val_dataset = datasets.ImageFolder(str(val_dir), transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

class_names = train_dataset.classes
print(f"Classes: {class_names}")


Classes: ['4011', '4015', '4088', '4196', '7020097009819', '7020097026113', '7023026089401', '7035620058776', '7037203626563', '7037206100022', '7038010009457', '7038010013966', '7038010021145', '7038010054488', '7038010068980', '7039610000318', '7040513000022', '7040513001753', '7040913336684', '7044610874661', '7048840205868', '7071688004713', '7622210410337', '90433917', '90433924', '94011']


In [10]:
model = timm.create_model("efficientnet_b0", pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace classifier
model.classifier = nn.Linear(model.classifier.in_features, len(class_names))

# Only train classifier
for param in model.classifier.parameters():
    param.requires_grad = True

model = model.to(device)


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.classifier.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

def train_one_epoch(model, loader):
    model.train()
    total_loss, correct = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct += (out.argmax(1) == y).sum().item()
    return total_loss / len(loader), correct / len(loader.dataset)

def validate(model, loader):
    model.eval()
    total_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            total_loss += loss.item()
            correct += (out.argmax(1) == y).sum().item()
    return total_loss / len(loader), correct / len(loader.dataset)


In [12]:
best_val_acc = 0
epochs_no_improve = 0
patience = 5  # Feel free to adjust
early_stop = False

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_loss, val_acc = validate(model, val_loader)
    
    print(f"Epoch {epoch+1}/{num_epochs} - Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "efficientnet_best.pth")
        epochs_no_improve = 0  # Reset counter
        print("✅ Accuracy improved — model saved.")
    else:
        epochs_no_improve += 1
        print(f"⚠️ No improvement for {epochs_no_improve} epoch(s).")

    # Step the learning rate scheduler if you're using one
    scheduler.step()

    # Check if we should stop
    if epochs_no_improve >= patience:
        print("⛔ Early stopping triggered.")
        early_stop = True
        break

Epoch 1/20 - Train Acc: 0.2129, Val Acc: 0.4591
Current learning rate: 0.000100
Epoch 2/20 - Train Acc: 0.4919, Val Acc: 0.6182
Current learning rate: 0.000100
Epoch 3/20 - Train Acc: 0.6267, Val Acc: 0.7030
Current learning rate: 0.000100
Epoch 4/20 - Train Acc: 0.7202, Val Acc: 0.7758
Current learning rate: 0.000100
Epoch 5/20 - Train Acc: 0.7778, Val Acc: 0.8106
Current learning rate: 0.000100
Epoch 6/20 - Train Acc: 0.8226, Val Acc: 0.8258
Current learning rate: 0.000050
Epoch 7/20 - Train Acc: 0.8393, Val Acc: 0.8409
Current learning rate: 0.000050
Epoch 8/20 - Train Acc: 0.8512, Val Acc: 0.8439
Current learning rate: 0.000050
Epoch 9/20 - Train Acc: 0.8628, Val Acc: 0.8561
Current learning rate: 0.000050
Epoch 10/20 - Train Acc: 0.8775, Val Acc: 0.8606
Current learning rate: 0.000050
Epoch 11/20 - Train Acc: 0.8833, Val Acc: 0.8561
Current learning rate: 0.000025


KeyboardInterrupt: 

In [30]:
# Load best model and evaluate
model.load_state_dict(torch.load("efficientnet_quickstart.pth"))
model.eval()

all_preds, all_labels = [], []

with torch.no_grad():
    for x, y in val_loader:
        x = x.to(device)
        out = model(x)
        preds = out.argmax(1).cpu()
        all_preds.extend(preds)
        all_labels.extend(y)

print(classification_report(all_labels, all_preds, target_names=class_names))


               precision    recall  f1-score   support

         4011       1.00      0.62      0.77        24
         4015       0.77      0.84      0.80        49
         4088       0.81      0.94      0.87        36
         4196       0.89      0.98      0.93        48
7020097009819       0.65      1.00      0.79        37
7020097026113       1.00      0.31      0.47        13
7023026089401       1.00      0.90      0.95        20
7035620058776       0.00      0.00      0.00         6
7037203626563       0.00      0.00      0.00        10
7037206100022       0.74      0.97      0.84        33
7038010009457       1.00      0.79      0.88        14
7038010013966       0.88      0.88      0.88        33
7038010021145       1.00      0.86      0.92        14
7038010054488       0.87      0.54      0.67        24
7038010068980       0.91      0.97      0.94        33
7039610000318       0.88      0.96      0.92        24
7040513000022       0.76      0.93      0.84        28
704051300

/Users/havardvd/git/Cogito/hackathon/4_apr/AST-NGD-COGITO-HACKATHON-2025/.venv/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/havardvd/git/Cogito/hackathon/4_apr/AST-NGD-COGITO-HACKATHON-2025/.venv/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/havardvd/git/Cogito/hackathon/4_apr/AST-NGD-COGITO-HACKATHON-2025/.venv/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 